In [56]:
from trueskill import Rating, rate
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import json

In [57]:
def print_commanders(commanders, skill_estimate_k = 3):
    commanders = {k: {"rating": v, "skill estimate": skill_estimate(v, k = skill_estimate_k)} for k, v in commanders.items()}
    ratings = [v["skill estimate"] for v in commanders.values()]
    rankings = sorted(range(len(ratings)), key=lambda i: ratings[i], reverse=True)
    for i, rank in enumerate(rankings):
        print(f"{i + 1}. {list(commanders.keys())[rank]} - {list(commanders.values())[rank]}")

In [58]:
def skill_estimate(rating, k = 3):
    """
    Calculate the conservative skill estimate for a given rating.
    This is the mu value minus k times the sigma value.
    """
    return float(rating.mu) - k * float(rating.sigma)

In [59]:
with open("results.json") as results_file:
    results = json.load(results_file)

In [60]:
commanders = set()
players = set()

for match in results:
    commanders.update([winner["commander"] for winner in match["winners"]])
    commanders.update([loser["commander"] for loser in match["losers"]])
    players.update([winner["player"] for winner in match["winners"]])
    players.update([loser["player"] for loser in match["losers"]])

print(commanders)
print(players)


commanders = {commander: Rating() for commander in commanders}
players = {player: Rating() for player in players}

{'Ojer Axonil, Deepest Might', "Atraxa, Praetors' Voice", 'Zedruu the Greathearted', 'Aminatou, Veil Piercer', 'Gandalf, Westward Voyager', 'Clavileño, First of the Blessed', "Eshki, Temur's Roar", 'Valgavoth, Harrower of Souls', 'Dionus, Elvish Archdruid', 'Maha, Its Feathers Night', 'Kianne, Corrupted Memory', 'Lord Windgrace', 'Umbris, Fear Manifest', 'Frodo, Adventurous Hobbit // Sam, Loyal Attendant', 'Nekusar, the Mindrazer', 'Krenko, Mob Boss', 'The Jolly Balloon Man', 'Rakdos, Lord of Riots', 'Bria, Riptide Rogue', 'Hazel of the Rootbloom', "Temmet, Naktamun's Will", 'Tom Bobadil', 'Vnwxt, Verbose Host', 'Omo, Queen of Vesuva', 'Saruman, the White Hand'}
{'Julian', 'Simon', 'Sebastian'}


In [61]:
# commanders = players

In [62]:
print_commanders(commanders)

1. Ojer Axonil, Deepest Might - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
2. Atraxa, Praetors' Voice - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
3. Zedruu the Greathearted - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
4. Aminatou, Veil Piercer - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
5. Gandalf, Westward Voyager - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
6. Clavileño, First of the Blessed - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
7. Eshki, Temur's Roar - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
8. Valgavoth, Harrower of Souls - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
9. Dionus, Elvish Archdruid - {'rating': trueskill.Rating(mu=25.000, sigma=8.333), 'skill estimate': 0.0}
10. Maha, Its Feathers Night - {'rating':

In [63]:
commanders_vis = pd.DataFrame(columns=["Match", "Commander", "TrueSkill Rating", "TrueSkill StDev"])

for i, match in enumerate(results):
    # print("\nMatch ", i + 1, " on ", match["date"], ":", sep="")
    commander_winners = [winner["commander"] for winner in match["winners"]]
    commander_losers = [loser["commander"] for loser in match["losers"]]
    commanders_in_match = commander_winners + commander_losers
    
    # print("Ratings before match:")
    # print_commanders({commander: commanders[commander] for commander in commanders_in_match})

    # print("\nWinners:", end=" ")
    # for winner in commander_winners[:(len(commander_winners)-1)]:
    #     print(winner, end="; ")
    # print(commander_winners[-1])
    # print("Losers:", end=" ")
    # for loser in commander_losers[:(len(commander_losers)-1)]:
    #     print(loser, end="; ")
    # print(commander_losers[-1])
    
    teams = [{commander: commanders[commander]} for commander in commander_winners] + \
             [{commander: commanders[commander]} for commander in commander_losers]
    ranks = [0] * len(commander_winners) + [1] * len(commander_losers)
    result = rate(teams, ranks=ranks)

    for team in result:
        for commander in team:
            commanders[commander] = team[commander]

    # print("\nRatings after match:")
    # print_commanders({commander: commanders[commander] for commander in commanders_in_match})

    current_match = pd.DataFrame({
        "Match": i + 1,
        "Commander": commanders_in_match,
        "TrueSkill Rating": [commanders[commander].mu for commander in commanders_in_match],
        "TrueSkill StDev": [commanders[commander].sigma for commander in commanders_in_match]
    })

    if commanders_vis.empty:
        commanders_vis = current_match
    else:
        previous_matches = commanders_vis[(commanders_vis["Match"] == i) & (~commanders_vis["Commander"].isin(commanders_in_match))]
        previous_matches.loc[:,"Match"] = i + 1

        commanders_vis = pd.concat([commanders_vis, current_match, previous_matches], ignore_index=True)

In [64]:
print_commanders(commanders)

1. Lord Windgrace - {'rating': trueskill.Rating(mu=32.064, sigma=5.942), 'skill estimate': 14.236644655978836}
2. Vnwxt, Verbose Host - {'rating': trueskill.Rating(mu=24.280, sigma=3.586), 'skill estimate': 13.521248371081352}
3. Valgavoth, Harrower of Souls - {'rating': trueskill.Rating(mu=26.925, sigma=4.522), 'skill estimate': 13.358179375747474}
4. Saruman, the White Hand - {'rating': trueskill.Rating(mu=31.614, sigma=6.431), 'skill estimate': 12.320638257397857}
5. Ojer Axonil, Deepest Might - {'rating': trueskill.Rating(mu=26.656, sigma=4.904), 'skill estimate': 11.94444153356054}
6. Bria, Riptide Rogue - {'rating': trueskill.Rating(mu=25.374, sigma=4.632), 'skill estimate': 11.4771499201016}
7. Kianne, Corrupted Memory - {'rating': trueskill.Rating(mu=26.730, sigma=5.120), 'skill estimate': 11.369271470120268}
8. Nekusar, the Mindrazer - {'rating': trueskill.Rating(mu=24.474, sigma=4.393), 'skill estimate': 11.29468710622712}
9. Krenko, Mob Boss - {'rating': trueskill.Rating(mu=

In [65]:
def generate_color_palette(n, alpha=1.0):
    return [
        f"hsla({h}, 70%, 50%, {alpha})"
        for h in range(0, 360, int(360 / n))
    ]

In [66]:
commanders_vis['Match'] = commanders_vis['Match'].astype(int)
commanders_vis = commanders_vis.sort_values(['Commander', 'Match'])

fig = go.Figure()

commanders = commanders_vis['Commander'].unique()
base_colors = generate_color_palette(len(commanders), alpha=1.0)
band_colors = generate_color_palette(len(commanders), alpha=0.1)

# Get distinct colors for each commander using Plotly's qualitative color palette
color_map_line = dict(zip(commanders, base_colors))
color_map_band = dict(zip(commanders, band_colors))

for commander in commanders:
    data = commanders_vis[commanders_vis['Commander'] == commander].copy()
    data = data.sort_values('Match')

    color_line = color_map_line[commander]
    color_band = color_map_band[commander]
    # color_rgba = color.replace('rgb', 'rgba').replace(')', ', 0.15)')  # Transparent fill

    upper = data['TrueSkill Rating'] + data['TrueSkill StDev']
    lower = data['TrueSkill Rating'] - data['TrueSkill StDev']

    # Line trace
    fig.add_trace(go.Scatter(
        x=data['Match'], y=data['TrueSkill Rating'],
        mode='lines',
        name=commander,
        legendgroup=commander,
        line=dict(width=2, color=color_line),
        hoverinfo='x+y+name'
    ))

    # Confidence band
    fig.add_trace(go.Scatter(
        x=pd.concat([data['Match'], data['Match'][::-1]]),
        y=pd.concat([upper, lower[::-1]]),
        fill='toself',
        fillcolor=color_band,
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo='skip',
        showlegend=False,
        legendgroup=commander
    ))

fig.update_layout(
    title='Commander TrueSkill Ratings with Confidence Bands',
    xaxis_title='Match',
    yaxis_title='TrueSkill Rating',
    template='plotly_white',
    legend_title='Commander',
    hovermode='x unified'
)

fig.show()

In [67]:
k = 3

commanders_vis["estimated skill"] = commanders_vis["TrueSkill Rating"] - 3 * commanders_vis["TrueSkill StDev"]

fig = go.Figure()

base_colors = generate_color_palette(len(commanders), alpha=1.0)

# Get distinct colors for each commander using Plotly's qualitative color palette
color_map_line = dict(zip(commanders, base_colors))

for commander in commanders:
    data = commanders_vis[commanders_vis['Commander'] == commander].copy()
    data = data.sort_values('Match')

    color_line = color_map_line[commander]

    # Line trace
    fig.add_trace(go.Scatter(
        x=data['Match'], y=data['estimated skill'],
        mode='lines',
        name=commander,
        legendgroup=commander,
        line=dict(width=2, color=color_line),
        hoverinfo='x+y+name'
    ))

fig.update_layout(
    title='Commander estimated Ratings ',
    xaxis_title='Match',
    yaxis_title='TrueSkill Rating',
    template='plotly_white',
    legend_title='Commander',
    hovermode='x unified'
)

fig.show()